In [1]:
import ee
import numpy as np
import pandas as pd
import geopandas as gpd
import geemap
import time
import utils_ee_Export_Phen

In [2]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [3]:
#read the csv file, which contains the US states' names and FIPS code
dfY = pd.read_csv('USID2.csv')
#set '0' as string
dfY['GEOID'] = dfY['GEOID'].astype(str)

## select the counties as featurColl

In [4]:
USCounties = ee.FeatureCollection('TIGER/2018/Counties')
# select the US counties by the 0 column in the dfY
countyCol = USCounties.filter(ee.Filter.inList('GEOID',dfY['GEOID'].tolist()))

## map the modis by soybean map and pheno stages

In [5]:
#calculate evi2
def addevi(image):
    evi = image.expression(
        '2.5 * float(nir - red)*0.0001 / float(nir*0.0001 + 2.4 * red*0.0001 + 1)',
        {
            'red': image.select('Nadir_Reflectance_Band1'),    # 620-670nm, RED
            'nir': image.select('Nadir_Reflectance_Band2'),    # 841-876nm, NIR
            #blue: image.select('Nadir_Reflectance_Band3')    # 459-479nm, BLUE
        });
    return image.addBands(evi.rename(['EVI2']))

In [6]:
# cancel all tasks in gee
# add a scolling bar for for loop
from tqdm import tqdm
from time import sleep

Country='US'

In [ ]:
#read the variable
variableList=['LST_Day_1km','LST_Night_1km']
Satellite_all = ee.ImageCollection("MODIS/061/MYD11A2")

for i in range(2):
  varName=variableList[i]
  for  yearInt in tqdm(range(2008,2019)):
    year=str(yearInt)
    #read the soybean map
    SoyMap=ee.Image('projects/ee-zcs/assets/CDL_500_MODIS/CDL_soybean_CB_'+year)

    Satellite = Satellite_all.filterDate(str(yearInt-1)+'-1-1',year+'-12-31').select(varName)

    utils_ee_Export_Phen.main(Satellite=Satellite, countyCol=countyCol, SoyMap=SoyMap, Country=Country,year=year, varName=varName,reducer='mean')


In [8]:
variableList=['EVI2']
Satellite_all = ee.ImageCollection('MODIS/061/MCD43A4')

for i in range(1):
  varName=variableList[i]
  for  yearInt in tqdm(range(2019,2022)):
    year=str(yearInt)
    #read the soybean map
    SoyMap=ee.Image('projects/ee-zcs/assets/CDL_500_MODIS/CDL_soybean_CB_'+year)

    # Satellite = Satellite_all.filterDate(str(yearInt-1)+'-1-1',year+'-12-31').select(varName)

    # varName='EVI2'
    Satellite = ee.ImageCollection('MODIS/061/MCD43A4').filterDate(str(yearInt-1)+'-1-1',year+'-12-31').map(addevi).select(varName)

    utils_ee_Export_Phen.main(Satellite=Satellite, countyCol=countyCol, SoyMap=SoyMap, Country=Country,year=year, varName=varName,reducer='mean')

100%|██████████| 3/3 [00:04<00:00,  1.51s/it]


## Read the data

In [27]:
# read geojosn file
pdts=gpd.read_file('My Drive/GEE_exports/ArgEVI2Hist.geojson')
pdts=pd.DataFrame(pdts)
pdts

,id,ADM0_ES,ADM0_PCODE,ADM1_ES,ADM1_PCODE,ADM2_ES,ADM2_PCODE,ADM2_REF,p1,p10,p20,p30,p40,p50,p60,p70,p80,p90,p99,geometry
0,00000000000000000047,Argentina,AR,Chaco,AR022,O'Higgins,AR022112,O'Higgins,0.360899,0.427825,0.458886,0.482480,0.505538,0.529092,0.552809,0.579890,0.611446,0.661768,0.764238,MULTIPOINT EMPTY
1,0000000000000000004e,Argentina,AR,Chaco,AR022,Independencia,AR022070,Independencia,0.412354,0.478526,0.517648,0.544978,0.576255,0.607134,0.642330,0.685465,0.736377,0.791115,0.877448,MULTIPOINT EMPTY
2,00000000000000000079,Argentina,AR,Chaco,AR022,Maipú,AR022091,Maipu,0.380825,0.435798,0.466938,0.486364,0.505645,0.525381,0.552644,0.580344,0.619099,0.669821,0.794468,MULTIPOINT EMPTY
3,00000000000000000083,Argentina,AR,Chaco,AR022,9 de Julio,AR022105,9 de Julio,0.365184,0.427592,0.462822,0.490256,0.517534,0.552754,0.591600,0.634724,0.693502,0.760117,0.872463,MULTIPOINT EMPTY
4,0000000000000000009d,Argentina,AR,Chaco,AR022,General Belgrano,AR022049,General Belgrano,0.381010,0.459376,0.502105,0.541123,0.579646,0.619387,0.661773,0.704860,0.755911,0.822133,0.888554,MULTIPOINT EMPTY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,000000000000000000d5,Argentina,AR,Santiago del Estero,AR086,Robles,AR086161,Robles,0.324984,0.379784,0.411098,0.432577,0.450206,0.469628,0.485247,0.506866,0.532137,0.574978,0.684040,MULTIPOINT EMPTY
201,00000000000000000141,Argentina,AR,Santiago del Estero,AR086,Aguirre,AR086007,Aguirre,0.294754,0.361308,0.392741,0.419883,0.447425,0.474231,0.501897,0.540816,0.580311,0.634976,0.747559,MULTIPOINT EMPTY
202,000000000000000001ac,Argentina,AR,Santiago del Estero,AR086,Banda,AR086035,Banda,0.329809,0.384957,0.412057,0.431161,0.450905,0.466666,0.486443,0.509909,0.544782,0.607545,0.794598,MULTIPOINT EMPTY
203,000000000000000001cf,Argentina,AR,Santiago del Estero,AR086,Copo,AR086056,Copo,0.409598,0.450211,0.477585,0.503214,0.534263,0.561051,0.590612,0.629841,0.678625,0.748830,0.846908,MULTIPOINT EMPTY
